# 🏦 Selector de Fondos de Inversión

Herramienta interactiva para filtrar y seleccionar los mejores fondos de inversión según el perfil del cliente.

## Características:
- **Filtros avanzados**: Por tipo de activo, región, divisa, riesgo, etc.
- **Sistema de scoring**: Puntuación personalizable según perfil de inversión
- **Perfiles predefinidos**: Conservador, Moderado, Agresivo, ESG, Largo Plazo
- **Visualizaciones interactivas**: Gráficos de riesgo-rendimiento, comparativas, etc.

In [ ]:
# Importar librerías necesarias
import pandas as pd
import numpy as np
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import warnings
warnings.filterwarnings('ignore')

# Importar módulos del proyecto
from src.data_processing import load_and_clean_data, get_column_descriptions
from src.filters import apply_filters, get_filter_options, filter_by_customer_profile
from src.scoring import calculate_fund_score, get_top_funds, PRESET_PROFILES, explain_score
from src.visualizations import (
    plot_risk_return_scatter,
    plot_top_funds_comparison,
    plot_fund_radar,
    plot_fees_comparison,
    create_fund_summary_table,
    plot_score_breakdown,
    plot_by_category
)

print("✅ Módulos cargados correctamente")

In [ ]:
# Cargar y limpiar datos
df = load_and_clean_data('funds.xlsx')
filter_options = get_filter_options(df)

print(f"📊 Fondos cargados: {len(df)}")
print(f"📋 Columnas disponibles: {len(df.columns)}")
print(f"\n🏷️ Tipos de activo: {filter_options['tipo_activo']}")
print(f"🌍 Regiones: {len(filter_options['region'])} disponibles")
print(f"💱 Divisas: {filter_options['divisa']}")

---
## 1️⃣ Perfil del Cliente

Configure el perfil de inversión del cliente:

In [ ]:
# === WIDGETS DEL PERFIL DEL CLIENTE ===

style = {'description_width': '150px'}
layout = widgets.Layout(width='400px')

# Inversión disponible
w_inversion = widgets.FloatText(
    value=5000,
    description='💰 Inversión (€):',
    style=style,
    layout=layout
)

# Horizonte temporal
w_horizonte = widgets.Dropdown(
    options=[('Corto plazo (< 2 años)', 'corto'), 
             ('Medio plazo (2-5 años)', 'medio'), 
             ('Largo plazo (> 5 años)', 'largo')],
    value='medio',
    description='⏱️ Horizonte:',
    style=style,
    layout=layout
)

# Tolerancia al riesgo
w_tolerancia = widgets.Dropdown(
    options=[('🛡️ Conservador - Prioriza seguridad', 'conservador'),
             ('⚖️ Moderado - Balance riesgo/rendimiento', 'moderado'),
             ('🚀 Agresivo - Maximiza rendimiento', 'agresivo')],
    value='moderado',
    description='📊 Tolerancia:',
    style=style,
    layout=layout
)

# Preferencia ESG
w_esg = widgets.Checkbox(
    value=False,
    description='🌱 Solo fondos sostenibles (ESG)',
    style=style,
    layout=layout
)

# Divisa preferida
w_divisa = widgets.Dropdown(
    options=[('Todas', None)] + [(d, d) for d in filter_options['divisa']],
    value=None,
    description='💱 Divisa:',
    style=style,
    layout=layout
)

# Perfil de scoring
w_perfil_scoring = widgets.Dropdown(
    options=[(PRESET_PROFILES[k]['nombre'], k) for k in PRESET_PROFILES.keys()],
    value='moderado',
    description='🎯 Perfil Score:',
    style=style,
    layout=layout
)

perfil_box = widgets.VBox([
    widgets.HTML('<h3>👤 Perfil del Cliente</h3>'),
    w_inversion,
    w_horizonte,
    w_tolerancia,
    w_esg,
    w_divisa,
    w_perfil_scoring
])

display(perfil_box)

---
## 2️⃣ Filtros Avanzados

Configure filtros adicionales para refinar la búsqueda:

In [ ]:
# === FILTROS AVANZADOS ===

# Tipo de activo
w_tipo_activo = widgets.SelectMultiple(
    options=filter_options['tipo_activo'],
    value=[],
    description='🏛️ Tipo activo:',
    style=style,
    layout=widgets.Layout(width='400px', height='120px')
)

# Región
w_region = widgets.SelectMultiple(
    options=filter_options['region'],
    value=[],
    description='🌍 Región:',
    style=style,
    layout=widgets.Layout(width='400px', height='150px')
)

# Nivel de riesgo
w_riesgo = widgets.IntRangeSlider(
    value=[1, 7],
    min=1,
    max=7,
    step=1,
    description='⚠️ Riesgo:',
    style=style,
    layout=layout
)

# Rating Morningstar mínimo
w_rating = widgets.IntSlider(
    value=1,
    min=1,
    max=5,
    step=1,
    description='⭐ Rating mín:',
    style=style,
    layout=layout
)

# Comisión TER máxima
w_ter_max = widgets.FloatSlider(
    value=3.0,
    min=0,
    max=5.0,
    step=0.1,
    description='💸 TER máx (%):',
    style=style,
    layout=layout
)

# Tipo de beneficio
w_beneficio = widgets.Dropdown(
    options=[('Todos', 'todos'), ('Solo Acumulado', 'acumulado'), ('Solo Distribuido', 'distribuido')],
    value='todos',
    description='📈 Beneficio:',
    style=style,
    layout=layout
)

filtros_box = widgets.VBox([
    widgets.HTML('<h3>🔍 Filtros Avanzados</h3>'),
    widgets.HBox([w_tipo_activo, w_region]),
    w_riesgo,
    w_rating,
    w_ter_max,
    w_beneficio
])

display(filtros_box)

---
## 3️⃣ Pesos Personalizados (Opcional)

Ajuste los pesos del sistema de scoring manualmente:

In [ ]:
# === PESOS PERSONALIZADOS ===

w_usar_custom = widgets.Checkbox(
    value=False,
    description='Usar pesos personalizados',
    style=style
)

peso_style = {'description_width': '180px'}
peso_layout = widgets.Layout(width='350px')

w_peso_rend12 = widgets.FloatSlider(value=0.15, min=0, max=0.5, step=0.05, 
                                     description='Rendimiento 12M:', style=peso_style, layout=peso_layout)
w_peso_rend36 = widgets.FloatSlider(value=0.15, min=0, max=0.5, step=0.05, 
                                     description='Rendimiento 36M:', style=peso_style, layout=peso_layout)
w_peso_rend60 = widgets.FloatSlider(value=0.10, min=0, max=0.5, step=0.05, 
                                     description='Rendimiento 60M:', style=peso_style, layout=peso_layout)
w_peso_sharpe = widgets.FloatSlider(value=0.20, min=0, max=0.5, step=0.05, 
                                     description='Ratio Sharpe:', style=peso_style, layout=peso_layout)
w_peso_riesgo = widgets.FloatSlider(value=0.10, min=0, max=0.5, step=0.05, 
                                     description='Bajo Riesgo:', style=peso_style, layout=peso_layout)
w_peso_comision = widgets.FloatSlider(value=0.15, min=0, max=0.5, step=0.05, 
                                       description='Bajas Comisiones:', style=peso_style, layout=peso_layout)
w_peso_rating = widgets.FloatSlider(value=0.10, min=0, max=0.5, step=0.05, 
                                     description='Rating Morningstar:', style=peso_style, layout=peso_layout)
w_peso_esg = widgets.FloatSlider(value=0.05, min=0, max=0.5, step=0.05, 
                                  description='Sostenibilidad ESG:', style=peso_style, layout=peso_layout)

# Output para mostrar suma de pesos
w_suma_pesos = widgets.HTML(value='<b>Suma de pesos: 1.00</b>')

def actualizar_suma(*args):
    suma = (w_peso_rend12.value + w_peso_rend36.value + w_peso_rend60.value + 
            w_peso_sharpe.value + w_peso_riesgo.value + w_peso_comision.value + 
            w_peso_rating.value + w_peso_esg.value)
    color = 'green' if abs(suma - 1.0) < 0.01 else 'red'
    w_suma_pesos.value = f'<b style="color:{color}">Suma de pesos: {suma:.2f}</b> (debe ser 1.00)'

for w in [w_peso_rend12, w_peso_rend36, w_peso_rend60, w_peso_sharpe, 
          w_peso_riesgo, w_peso_comision, w_peso_rating, w_peso_esg]:
    w.observe(actualizar_suma, 'value')

pesos_box = widgets.VBox([
    widgets.HTML('<h3>⚖️ Pesos Personalizados</h3>'),
    w_usar_custom,
    widgets.VBox([
        w_peso_rend12, w_peso_rend36, w_peso_rend60, w_peso_sharpe,
        w_peso_riesgo, w_peso_comision, w_peso_rating, w_peso_esg,
        w_suma_pesos
    ])
])

display(pesos_box)

---
## 4️⃣ Buscar Fondos

Ejecute la búsqueda con los filtros y perfil configurados:

In [ ]:
# === FUNCIÓN DE BÚSQUEDA ===

output_results = widgets.Output()

def buscar_fondos(b=None):
    with output_results:
        clear_output(wait=True)
        
        print("🔍 Buscando fondos...\n")
        
        # Construir filtros
        filters = {
            'inversion_cliente': w_inversion.value,
            'tolerancia_minimo': 0.1,
            'nivel_riesgo_min': w_riesgo.value[0],
            'nivel_riesgo_max': w_riesgo.value[1],
            'rating_min': w_rating.value,
            'comision_ter_max': w_ter_max.value / 100,
        }
        
        # Filtros opcionales
        if w_tipo_activo.value:
            filters['tipo_activo'] = list(w_tipo_activo.value)
        if w_region.value:
            filters['region'] = list(w_region.value)
        if w_divisa.value:
            filters['divisa'] = [w_divisa.value]
        if w_esg.value:
            filters['solo_sostenibles'] = True
        if w_beneficio.value == 'acumulado':
            filters['solo_acumulado'] = True
        elif w_beneficio.value == 'distribuido':
            filters['solo_distribuido'] = True
        
        # Aplicar filtros
        filtered_df = apply_filters(df, filters)
        
        print(f"📊 Fondos después de filtrar: {len(filtered_df)} de {len(df)}")
        
        if len(filtered_df) == 0:
            print("\n⚠️ No se encontraron fondos con estos criterios. Intenta relajar los filtros.")
            return
        
        # Calcular scores
        if w_usar_custom.value:
            custom_weights = {
                'rendimiento_12m': w_peso_rend12.value,
                'rendimiento_36m': w_peso_rend36.value,
                'rendimiento_60m': w_peso_rend60.value,
                'sharpe_ratio': w_peso_sharpe.value,
                'riesgo_bajo': w_peso_riesgo.value,
                'comisiones_bajas': w_peso_comision.value,
                'rating_morningstar': w_peso_rating.value,
                'rating_sostenibilidad': w_peso_esg.value,
            }
            scored_df = calculate_fund_score(filtered_df, custom_weights=custom_weights)
        else:
            scored_df = calculate_fund_score(filtered_df, weights=w_perfil_scoring.value)
        
        # Obtener top 10
        top_10 = scored_df.head(10)
        
        # Guardar en variable global para visualizaciones
        global resultado_busqueda
        resultado_busqueda = scored_df
        
        # Mostrar tabla resumen
        print("\n" + "="*80)
        print("🏆 TOP 10 FONDOS RECOMENDADOS")
        print("="*80 + "\n")
        
        tabla = create_fund_summary_table(top_10)
        display(tabla)
        
        # Mostrar perfil usado
        if not w_usar_custom.value:
            perfil = PRESET_PROFILES[w_perfil_scoring.value]
            print(f"\n📋 Perfil de scoring: {perfil['nombre']}")
            print(f"   {perfil['descripcion']}")

# Botón de búsqueda
btn_buscar = widgets.Button(
    description='🔍 Buscar Fondos',
    button_style='primary',
    layout=widgets.Layout(width='200px', height='40px')
)
btn_buscar.on_click(buscar_fondos)

display(btn_buscar)
display(output_results)

---
## 5️⃣ Visualizaciones

Gráficos interactivos de los resultados:

In [ ]:
# === GRÁFICO RIESGO VS RENDIMIENTO ===

try:
    if 'resultado_busqueda' in dir() and len(resultado_busqueda) > 0:
        fig = plot_risk_return_scatter(
            resultado_busqueda,
            title='Relación Riesgo - Rendimiento (Fondos Filtrados)',
            highlight_funds=resultado_busqueda.head(10)['fund_id'].tolist()
        )
        fig.show()
    else:
        print("⚠️ Primero ejecuta la búsqueda de fondos (celda anterior)")
except NameError:
    print("⚠️ Primero ejecuta la búsqueda de fondos (celda anterior)")

In [ ]:
# === COMPARACIÓN TOP 10 ===

try:
    if 'resultado_busqueda' in dir() and len(resultado_busqueda) > 0:
        fig = plot_top_funds_comparison(resultado_busqueda, n=10)
        fig.show()
    else:
        print("⚠️ Primero ejecuta la búsqueda de fondos")
except NameError:
    print("⚠️ Primero ejecuta la búsqueda de fondos")

In [ ]:
# === DESGLOSE DEL SCORE ===

try:
    if 'resultado_busqueda' in dir() and len(resultado_busqueda) > 0:
        fig = plot_score_breakdown(resultado_busqueda, n=10)
        if fig:
            fig.show()
    else:
        print("⚠️ Primero ejecuta la búsqueda de fondos")
except NameError:
    print("⚠️ Primero ejecuta la búsqueda de fondos")

In [ ]:
# === COMPARACIÓN DE COMISIONES ===

try:
    if 'resultado_busqueda' in dir() and len(resultado_busqueda) > 0:
        fig = plot_fees_comparison(resultado_busqueda, n=10)
        fig.show()
    else:
        print("⚠️ Primero ejecuta la búsqueda de fondos")
except NameError:
    print("⚠️ Primero ejecuta la búsqueda de fondos")

---
## 6️⃣ Detalle de un Fondo Específico

Analiza en detalle un fondo del top 10:

In [ ]:
# === SELECTOR DE FONDO ESPECÍFICO ===

output_detalle = widgets.Output()

def mostrar_detalle_fondo(idx):
    with output_detalle:
        clear_output(wait=True)
        
        if 'resultado_busqueda' not in dir() or len(resultado_busqueda) == 0:
            print("⚠️ Primero ejecuta la búsqueda de fondos")
            return
        
        if idx < 0 or idx >= len(resultado_busqueda):
            print("⚠️ Índice fuera de rango")
            return
        
        fondo = resultado_busqueda.iloc[idx]
        
        print("\n" + "="*80)
        print(f"📋 DETALLE DEL FONDO: {fondo['fund_name']}")
        print("="*80)
        
        print(f"\n🏢 Gestora: {fondo['fund_manager']}")
        print(f"📄 ISIN: {fondo['isin']}")
        print(f"🎯 Score: {fondo['score']:.1f}/100")
        print(f"\n📊 CARACTERÍSTICAS:")
        print(f"   • Tipo de activo: {fondo['Tipo de activo']}")
        print(f"   • Región: {fondo['Región']}")
        print(f"   • Divisa: {fondo['Divisa']}")
        print(f"   • Nivel de riesgo: {fondo['Nivel de riesgo_clean']}/7")
        print(f"   • Rating Morningstar: {fondo['Rating Morningstar']} ⭐")
        print(f"   • Sostenible (ESG): {'Sí ✅' if fondo['es_sostenible'] else 'No'}")
        
        print(f"\n💰 RENDIMIENTOS:")
        print(f"   • Año actual: {fondo['Ren. año actual']}")
        print(f"   • Últimos 12 meses: {fondo['Ren. últ. 12 meses']}")
        print(f"   • Últimos 36 meses: {fondo['Ren. últ. 36 meses']}")
        print(f"   • Últimos 60 meses: {fondo['Ren. últ. 60 meses']}")
        
        print(f"\n💸 COMISIONES:")
        print(f"   • TER: {fondo['Comisión TER']}")
        print(f"   • Gestión: {fondo['Comisión gestión']}")
        print(f"   • Suscripción: {fondo['Comisión suscripción']}")
        print(f"   • Reembolso: {fondo['Comisión reembolso']}")
        
        print(f"\n📈 MÉTRICAS DE RIESGO:")
        print(f"   • Sharpe Ratio: {fondo['Sharpe Ratio']}")
        print(f"   • Beta: {fondo['Beta']}")
        print(f"   • Máxima caída: {fondo['Máxima caída del fondo']}")
        
        print(f"\n💵 INVERSIÓN MÍNIMA: {fondo['min_first_buy']}")
        
        # Gráfico radar del fondo
        try:
            fig = plot_fund_radar(fondo, weights=w_perfil_scoring.value)
            fig.show()
        except:
            pass

# Selector de fondo
w_selector_fondo = widgets.IntSlider(
    value=0,
    min=0,
    max=9,
    step=1,
    description='Fondo #:',
    style=style
)

btn_detalle = widgets.Button(
    description='📋 Ver Detalle',
    button_style='info',
    layout=widgets.Layout(width='150px')
)
btn_detalle.on_click(lambda b: mostrar_detalle_fondo(w_selector_fondo.value))

display(widgets.HBox([w_selector_fondo, btn_detalle]))
display(output_detalle)

---
## 7️⃣ Análisis Exploratorio General

Estadísticas generales del universo de fondos:

In [ ]:
# === ESTADÍSTICAS POR TIPO DE ACTIVO ===

fig = plot_by_category(df, 'Tipo de activo', 'Ren. últ. 12 meses_clean', 
                       agg='mean', title='Rendimiento Promedio 12M por Tipo de Activo (%)')
fig.show()

In [ ]:
# === ESTADÍSTICAS POR REGIÓN ===

fig = plot_by_category(df, 'Región', 'Ren. últ. 12 meses_clean', 
                       agg='mean', title='Rendimiento Promedio 12M por Región (%)')
fig.show()

In [ ]:
# === DISTRIBUCIÓN DE NIVELES DE RIESGO ===

import plotly.express as px

risk_counts = df['Nivel de riesgo_clean'].value_counts().sort_index()
fig = px.bar(x=risk_counts.index, y=risk_counts.values, 
             title='Distribución de Fondos por Nivel de Riesgo',
             labels={'x': 'Nivel de Riesgo', 'y': 'Número de Fondos'})
fig.show()

In [ ]:
# === SCATTER GENERAL: TODOS LOS FONDOS ===

fig = plot_risk_return_scatter(df, title='Universo Completo: Riesgo vs Rendimiento 12M')
fig.show()

---
## 8️⃣ Exportar Resultados

In [ ]:
# === EXPORTAR TOP FONDOS A CSV ===

def exportar_resultados(filename='fondos_recomendados.csv'):
    if 'resultado_busqueda' not in dir():
        print("⚠️ Primero ejecuta la búsqueda de fondos")
        return
    
    cols_export = [
        'fund_name', 'fund_manager', 'isin', 'score',
        'Tipo de activo', 'Región', 'Divisa',
        'Nivel de riesgo_clean', 'Rating Morningstar',
        'Ren. últ. 12 meses', 'Ren. últ. 36 meses',
        'Comisión TER', 'Sharpe Ratio',
        'min_first_buy', 'es_sostenible'
    ]
    
    export_df = resultado_busqueda.head(50)[cols_export]
    export_df.to_csv(filename, index=False, encoding='utf-8-sig')
    print(f"✅ Resultados exportados a: {filename}")

# Botón de exportar
btn_exportar = widgets.Button(
    description='💾 Exportar Top 50 a CSV',
    button_style='success',
    layout=widgets.Layout(width='200px')
)
btn_exportar.on_click(lambda b: exportar_resultados())

display(btn_exportar)